In [362]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
import astropy.coordinates as coord
from astropy.table import Table
from astropy.io import ascii
from scipy.optimize import fmin_cg

In [363]:
#Initiation of data, now loading GRD1 data and setting up the various parameters used in the code below

def load_data(file,flagset):
    
    data_raw = ascii.read(file, format='fast_csv')
    
    data = Table(data_raw, copy=True)
    
    if flagset == 'flag_dup':
        
        flag_list=['flag_dup']
        
    elif flagset == 'flag_any':
        
        flag_list=['flag_any']

    elif flagset == 'flag_any-lowlogg':
        
        flag_list=['flag_dup','flag_N','flag_outlier','flag_pole']
        
    elif flagset == 'None':
        
        flag_list = []
        
    elif flagset == 'custom':
        
        flag_list=str(input('Tell me which flags to remove: ')).split(',')

    else:
        raise Exception('Not a valid flagset')
    
    bad_rows = np.array([])
    
    for i in flag_list:
        
        bad = np.nonzero(data[i])
        
        bad_rows = np.concatenate((bad_rows,bad[0]))
    
    bad_rows = np.unique(bad_rows)
    
    data.remove_rows(bad_rows.astype(int))
    
    return data,data_raw


flagset = 'flag_any'

try:
    data
except NameError:
    
    data,data_raw = load_data('Distances_PJM2017.csv',flagset)

data = data_raw[0:10000]
    
RA = data['RAdeg']*u.degree
DEC = data['DEdeg']*u.degree
pm_RA = data['pmRA_TGAS']*u.mas/u.yr
pm_DEC = data['pmDE_TGAS']*u.mas/u.yr
parallax = data['parallax']*u.mas

sample = coord.ICRS(ra = RA, dec = DEC, pm_ra_cosdec = pm_RA, pm_dec = pm_DEC)

sample = sample.transform_to(coord.Galactic)

#Oort constant values from Bovy (2018)
A = (15.3*(u.km/(u.s*u.kpc))).to(1/u.yr)
B = (-11.9*(u.km/(u.s*u.kpc))).to(1/u.yr)

mul_obs = sample.pm_l_cosb.to(1/u.yr,equivalencies = u.dimensionless_angles())
mub_obs = sample.pm_b.to(1/u.yr,equivalencies = u.dimensionless_angles())

bvals = sample.b.to(u.deg)
lvals = sample.l.to(u.deg)

"""Computation of the relevant quantities

    l,b: Galactic coordinates
    s: the distance obtained by inverting the parallax
    mul, mub: proper motion in l and b
    pvals: Tangential velocities obtained from eq. 2 in DB98
    rhatvals: The unit vector of each star
    vmin: Vector containing the minimum velocities in v-space
    n: The number of cells we want in each dimension of our v-space box
    dv: Step sizes for each dimension"""

b = np.deg2rad(bvals).value # just a test
l = np.deg2rad(lvals).value
cosl = np.cos(l)
cosb = np.cos(b)
sinl = np.sin(l)
sinb = np.sin(b)
s = parallax.to(u.kpc,equivalencies=u.parallax())

mul = mul_obs - A*np.cos(2*l)-B
mub = mub_obs + A*np.sin(2*l)*cosb*sinb

pvals = s*np.array([-sinl*cosb*mul - cosl*sinb*mub,
                 cosl*cosb*mul - sinl*sinb*mub,
                 cosb*mub])/u.yr
    
rhatvals = np.array([cosb*cosl, cosb*sinl, sinb]).T
pvals = pvals.to(u.km/u.s).value.T

"""Test values for our functions"""

#Some dispersions and mean velocities for Gaussians
thin0 = np.array([0,215,0])
thick0 = np.array([0,180,0])
halo0 = np.array([0,0,0])
thin_disp  = np.array([30,20,17])
thick_disp  = np.array([80,60,55])
halo_disp = np.array([160,100,100])

vmin = np.array([-200,-200,-200])

n = np.array([8,8,8])

dv = np.array([50,50,50])

In [364]:
def calc_K(pk,rhat,vmin,dv,n):
    '''Calculate the values of K simultaneously for all bins for a given star with p, rhat'''
    
    vxmin, vymin, vzmin = vmin
    dvx, dvy, dvz = dv
    nx, ny, nz = n
    pkx, pky, pkz = pk
    rhatx, rhaty, rhatz = rhat
    
    K = np.zeros((nx,ny,nz))
    
    vxmax, vymax, vzmax = vxmin+nx*dvx,vymin+ny*dvy,vzmin+nz*dvz
    
    """We now solve the line equation v = pk + vr*rhat, where v are the intersections of the line 
    and the boundaries of each bin"""
    
    vx_bins = np.arange(vxmin, vxmax+dvx, dvx)
    vy_bins = np.arange(vymin, vymax+dvy, dvy)
    vz_bins = np.arange(vzmin, vzmax+dvz, dvz)
    
    if np.round(np.linalg.norm(rhat))!=1:
        raise ValueError('rhat must be a unit vector')
    
    vrx = (vx_bins-pkx)/rhatx
    vry = (vy_bins-pky)/rhaty
    vrz = (vz_bins-pkz)/rhatz
    
    """After solving the line equation for each dim we remove the vr values which solve the equation
    for bins outside of our specified box."""
    
    vrmax = min(max(vrx),max(vry),max(vrz))
    vrmin = max(min(vrx),min(vry),min(vrz))
    
    vrx = vrx[(vrx<=vrmax) & (vrx>=vrmin)]
    vry = vry[(vry<=vrmax) & (vry>=vrmin)]
    vrz = vrz[(vrz<=vrmax) & (vrz>=vrmin)]
    vr = np.concatenate((vrx,vry,vrz))
    vr.sort() #We obtain an ordered list with all vr values for intersections between entry and exit points
    
    if len(vr)==0:
        return K
    
    vr_prime =(vr[:-1] + vr[1:]) / 2
    line_bins = np.zeros((len(vr_prime),3))

    pk = np.stack([pk]*len(vr_prime))
    rhat = np.stack([rhat]*len(vr_prime))
    vmin = np.stack([vmin]*len(vr_prime))
    vr_primestack = np.stack([vr_prime]*3,axis=1)

    """We now solve the line equation again for values in the middle of each bin with a line segment in it.
    This gives us the coordinates for each relevant bin, given in line_bins.
    Finally we computhe the length of each segment and add said value to the relevant box in our K-space."""
    v_prime = pk + vr_primestack*rhat
    line_bins += np.floor((v_prime-vmin)/ dv)
      
    line_bins = line_bins.astype(int)
    
    line_len = vr[1:]-vr[:-1]
    non_zero = np.nonzero(line_len)
    line_len = line_len[non_zero]
    line_bins = line_bins[non_zero]
    
    K[line_bins[:,0],line_bins[:,1],line_bins[:,2]] = line_len/(dvx*dvy*dvz)
    
    return K

In [365]:
def calc_sigma2(pvals,rhat):
    
    """Function that applies equation 12 of DB98 for a set of stars from their tangential velocities and unit vectors.
    Returns the velocity dispersion tensor."""
    
    pmean = np.mean(pvals, axis=0)
    
    rhat_outer = rhat[:,:,None]*rhat[:,None,:] #Fast way of getting the outer product for each rhat with itself.

    iden = np.identity(3)
    
    A = np.stack([iden]*len(rhat_outer))-rhat_outer #Eq. 4 in DB98. Yields an array of dim (N_star,3,3)
    
    A_mean = np.mean(A,axis=0)
    A_mean_inv = np.linalg.inv(A_mean)
    v_mean = np.dot(A_mean_inv, pmean)
    
    pp = pvals - np.dot(A,v_mean)
    
    pp2mean = np.mean(pp*pp,axis=0)
    
    B = np.array([[9,-1,-1],[-1,9,-1],[-1,-1,9]])
    
    sigma2 = (3/14)*np.dot(B,pp2mean)
    
    return sigma2

In [366]:
def nl_delta(n,l):
    
    """Checks if our given vector n is within one unit vector e_i of the cell l
    
    First attempt at solving the problem, so can be ignored"""
    
    e_x = np.array([1,0,0])
    e_y = np.array([0,1,0])
    e_z = np.array([0,0,1])
    
    rules = [np.array_equal(n,l+e_x),
            np.array_equal(n,l-e_x),
            np.array_equal(n,l+e_y),
            np.array_equal(n,l-e_y),
            np.array_equal(n,l+e_z),
            np.array_equal(n,l-e_z)]
    
    if np.array_equal(n,l):
        delta = -2
    elif any(rules):
        delta = 1
    else:
        delta = 0
    
    return delta
        
def calc_xhi(line_bins,sigma2,hx,hy,hz,nx,ny,nz):

    """Given a vector l, find the estimate of the second derivative. Compare l with possible adjacent n values.
    
    Was also scrapped for a more efficient method sec_der"""   
    
    h2 = np.array([hx**2,hy**2,hz**2])
    
    xhi = np.zeros((len(line_bins),7))
    
    n_bins = np.array([nx,ny,nz])
    
    e_x = np.array([1,0,0])
    e_y = np.array([0,1,0])
    e_z = np.array([0,0,1])
    
    for i in range(len(line_bins)):
        
        l = line_bins[i]
        
        n_list = [l,
                 l+e_x,l-e_x,
                 l+e_y,l-e_y,
                 l+e_z,l-e_z]
        
        print(n_list)
        
        for j in range(7):
            
            n = n_list[j]
            
            if (all(n>=0)) and (all(n<=n_bins)):
                xhi[i][j] += np.sum((sigma2/h2) *  nl_delta(n,l))

    return xhi

In [367]:
def sec_der(phi,sigma2,dv):
    
    """Estimates the second deriative for ln(f(v_l)) given a sample of stars (eq. 30 of D98).
    Takes contributions at the phi values of adjacent bins for each bin l.
    
    We create a new, larger box with dimensions n+2 centred on our phi-space box.
    This allows us to disregard any issues at the bins at the boundaries of our phi-space."""
    
    nx, ny, nz = phi.shape
    dv2 = dv**2
    
    nxx, nyy, nzz = nx+2, ny+2, nz+2 

    phip = np.zeros((nxx,nyy,nzz)) #new larger box

    phip[1:-1,1:-1,1:-1] = phi #puts the phi-box in the centre of our larger box
    
    kappa = sigma2/dv2
    
    kappa_sum = -2*sum(sigma2/dv2)
    
    """Here we compute the contributions from all the adjacent bins simultaneously.
    In every dimension we sum the phi values of box l-1 and l+1 and multiply with the relevant factor"""
    
    phi_fac = np.array([phip[0:nxx-2,1:-1,1:-1]+phip[2:nxx,1:-1,1:-1],
                           phip[1:-1,0:nyy-2,1:-1]+phip[1:-1,2:nyy,1:-1],
                           phip[1:-1,1:-1,0:nzz-2]+phip[1:-1,1:-1,2:nzz]])

    phi_arrx = (sigma2[0]/dv2[0])*phi_fac[0]
    phi_arry = (sigma2[1]/dv2[1])*phi_fac[1]
    phi_arrz = (sigma2[2]/dv2[2])*phi_fac[2]
    
    """We sum all contributions from adjacent boxes and finally add the terms for each box l. 
    Yields a box with the same dimensions as phi, containing the second derivative values for each bin."""
    
    phi_arr = phi_arrx+phi_arry+phi_arrz+kappa_sum*phi 

    return phi_arr

In [368]:
def phi_guess(v0,disp,vmin,dv,n):
    
    """Provides an initial guess of the phi values in each bin. For now only allows for a Gaussian type guess given arrays
    with mean velocities and dispersions for each dimension."""
    
    vxmin, vymin, vzmin = vmin
    dvx, dvy, dvz = dv
    nx, ny, nz = n
    
    vxmax, vymax, vzmax = vxmin+nx*dvx,vymin+ny*dvy,vzmin+nz*dvz
    
    vx_bins = np.arange(vxmin, vxmax+dvx, dvx)
    vy_bins = np.arange(vymin, vymax+dvy, dvy)
    vz_bins = np.arange(vzmin, vzmax+dvz, dvz)
    
    vxc = (vx_bins[1:]+vx_bins[:-1])/2
    vyc = (vy_bins[1:]+vy_bins[:-1])/2
    vzc = (vz_bins[1:]+vz_bins[:-1])/2
    
    v0 = np.stack([v0]*len(vxc))
    v0x = v0[:,0]
    v0y = v0[:,1]
    v0z = v0[:,2]
    
    disp = np.stack([disp]*len(vxc))
    dispx = disp[:,0]
    dispy = disp[:,1]
    dispz = disp[:,2]
    
    """Given the velocities of each bin we compute the 3D Gaussian value."""
    
    gx = np.exp(-((vxc-v0x)**2)/(2*dispx**2)) / (dispx*np.sqrt(2*np.pi))
    gy = np.exp(-((vyc-v0y)**2)/(2*dispy**2)) / (dispy*np.sqrt(2*np.pi))
    gz = np.exp(-((vzc-v0z)**2)/(2*dispz**2)) / (dispz*np.sqrt(2*np.pi))
    
    phix, phiy, phiz = np.meshgrid(gx,gy,gz)
    
    phi = np.array([phix,phiy,phiz])
    
    phi = np.sum(phi,axis=0)
    
    return phi.T

In [369]:
def get_L(phi,*args):
    
    """The function that we wish to optimise."""
    
    K, N, alpha, dv, n, sigma2 = args
    
    dvx, dvy, dvz = dv
    
    phi = np.reshape(phi,n)
    
    phixhi = sec_der(phi,sigma2,dv)
    
    exphi = np.exp(phi)
    
    Kphi = exphi*K
    K_sum = np.sum(np.log(Kphi),axis=0)
        
    L_tilde = K_sum/N + np.sum(exphi)-(alpha/(2*dvx*dvy*dvz))*np.sum(phixhi**2)
    
    negL = -1*(L_tilde+1)
    
    return negL

In [370]:
def max_L(alpha, pvals, rhatvals, vmin, dv, n):
    
    """Function that employs scipy.optimize.fmin_cg to maximise the function get_L()"""
    
    thin0 = np.array([0,215,0]) #Just a guess of the Gaussian
    thin_disp  = np.array([30,20,17])
    
    dvx, dvy, dvz = dv
    nx, ny, nz = n
    
    N = len(pvals)
    
    phi0 = phi_guess(thin0,thin_disp,vmin,dv,n)
    
    sigma2 = calc_sigma2(pvals,rhatvals)
    
    Kvals = np.zeros((N,nx,ny,nz))
    
    for i in range(N):
        K = calc_K(pvals[i],rhatvals[i],vmin,dv,n)
        Kvals[i] += K
        
    args = (K, N, alpha, dv, n, sigma2)
    
    phi0 = np.ravel(phi0)
    
    #Currently dealing with an issue where fmin_cg cannot numerically estimate the gradient of our function
    
    fmin_cg(get_L, phi0,args=args)

In [371]:
max_L(1, pvals, rhatvals, vmin, dv, n) #Does not work yet

C:\Users\Joni\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log
  app.launch_new_instance()
C:\Users\Joni\Anaconda3\lib\site-packages\scipy\optimize\optimize.py:628: RuntimeWarning: invalid value encountered in subtract
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]


ValueError: setting an array element with a sequence.

In [ ]:
#def max_L(phi,*args):
    
    #make initial guess of phi (write new phi func)
    #maximize for these two parameters given a sample with stars using scipy.optimize.fmin_cg
    #prosper

In [ ]:
#%timeit calc_sigma2(pvals,rhatvals)